### Import libraries :

In [6]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

### Load Preprocessed Data :

In [4]:
dataframe = pd.read_csv('processed_train_data.csv')
dataset=dataframe.values

#inputs:
X = dataset[:,0:6].astype(float)
X=pd.DataFrame(X)

#targets:
Y_breed = dataset[:,6:7]
Y_pet = dataset[:,7]

#test data:
test_data = pd.read_csv('processed_test_data.csv')
test_data.head()

,Date Difference,condition,color_encoded,Size,X1,X2
0,4405.0,0.0,2,0.371751,0,7
1,175.0,1.0,36,0.004026,0,1
2,2000.0,1.0,2,0.098904,0,7
3,1148.0,1.0,2,0.024534,7,1
4,464.0,1.0,14,0.219532,0,7


### Create target dummies for breed and pet :

In [5]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
encoder.fit(Y_breed.ravel())
encoder_Y = encoder.transform(Y_breed.ravel())

dummy_b= pd.get_dummies(encoder_Y)
dummy_b.head()

,0,1,2
0,1,0,0
1,1,0,0
2,0,0,1
3,1,0,0
4,1,0,0


In [ ]:
Y_pet = np.where(Y_pet == 4,3,Y_pet)
Y_pet

array([1., 2., 3., ..., 1., 2., 2.])

In [ ]:
encoder = LabelEncoder()
encoder.fit(Y_pet.ravel())
encoder_Y = encoder.transform(Y_pet.ravel())

dummy_p = pd.get_dummies(encoder_Y)
dummy_p.head()

,0,1,2,3
0,0,1,0,0
1,0,0,1,0
2,0,0,0,1
3,0,0,1,0
4,0,1,0,0


### Model + Training :

##### Model for breed_category :

In [ ]:
# define baseline model
def baseline_model():
    # create model
  model = Sequential()
  model.add(Dense(50, input_dim=6, activation='relu'))
  model.add(Dense(60, activation='relu'))
  model.add(Dense(60, activation='relu'))
  model.add(Dense(3, activation='softmax'))
  # Compile model
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

estimator_b = KerasClassifier(build_fn=baseline_model, epochs=500, batch_size=80, verbose=1)
kfold = KFold(n_splits=2, shuffle=True)
estimator_b.fit(X,dummy_b, verbose=1)

results = cross_val_score(estimator_b, X, dummy_b, cv=kfold)
# print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Epoch 1/500
236/236 [==============================] - 0s 2ms/step - loss: 2.7620 - accuracy: 0.4572
Epoch 2/500
236/236 [==============================] - 0s 1ms/step - loss: 1.9040 - accuracy: 0.4917
Epoch 3/500
236/236 [==============================] - 0s 1ms/step - loss: 1.4768 - accuracy: 0.5256
Epoch 4/500
236/236 [==============================] - 0s 1ms/step - loss: 1.2446 - accuracy: 0.5656
Epoch 5/500
236/236 [==============================] - 0s 2ms/step - loss: 1.2597 - accuracy: 0.5905
Epoch 6/500
236/236 [==============================] - 0s 2ms/step - loss: 1.1078 - accuracy: 0.6290
Epoch 7/500
236/236 [==============================] - 0s 1ms/step - loss: 0.9788 - accuracy: 0.6564
Epoch 8/500
236/236 [==============================] - 0s 2ms/step - loss: 0.9124 - accuracy: 0.6795
Epoch 9/500
236/236 [==============================] - 0s 2ms/step - loss: 0.8161 - accuracy: 0.7072
Epoch 10/500
236/236 [==============================] - 0s 2ms/step - loss: 0.8434 - accura

In [ ]:
estimator_b.score(X,dummy_b)

236/236 [==============================] - 0s 893us/step - loss: 0.1880 - accuracy: 0.9019


0.901879608631134

#### Predict breed of test data :

In [ ]:
y_hat1=estimator_b.predict(test_data)
y_hat1


101/101 [==============================] - 0s 788us/step


array([1, 0, 0, ..., 1, 0, 1])

In [ ]:
y_b = pd.DataFrame(y_hat1)
y_b[0].unique()

array([1, 0, 2])

In [ ]:
cols = ['breed_category']
y_b.columns = cols
y_b

,breed_category
0,1
1,0
2,0
3,0
4,0
...,...
8067,1
8068,1
8069,1
8070,0


#### Model for pet_category :

In [ ]:
def baseline_model1():
    # create model
  model = Sequential()
  model.add(Dense(70, input_dim=6, activation='relu'))
  model.add(Dense(70, activation='relu'))
  model.add(Dense(4, activation='softmax'))
  # Compile model
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

estimator_p = KerasClassifier(build_fn=baseline_model1, epochs=300, batch_size=80, verbose=1)
kfold = KFold(n_splits=2, shuffle=True)
estimator_p.fit(X,dummy_p)
results = cross_val_score(estimator_p, X, dummy_p, cv=kfold)
# print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Epoch 1/300
236/236 [==============================] - 0s 1ms/step - loss: 3.7850 - accuracy: 0.5855
Epoch 2/300
236/236 [==============================] - 0s 1ms/step - loss: 1.6533 - accuracy: 0.6330
Epoch 3/300
236/236 [==============================] - 0s 1ms/step - loss: 1.7794 - accuracy: 0.6365
Epoch 4/300
236/236 [==============================] - 0s 1ms/step - loss: 1.4298 - accuracy: 0.6578
Epoch 5/300
236/236 [==============================] - 0s 1ms/step - loss: 1.4249 - accuracy: 0.6601
Epoch 6/300
236/236 [==============================] - 0s 1ms/step - loss: 1.9721 - accuracy: 0.6452
Epoch 7/300
236/236 [==============================] - 0s 1ms/step - loss: 1.4078 - accuracy: 0.6572
Epoch 8/300
236/236 [==============================] - 0s 1ms/step - loss: 1.2723 - accuracy: 0.6644
Epoch 9/300
236/236 [==============================] - 0s 1ms/step - loss: 1.3238 - accuracy: 0.6609
Epoch 10/300
236/236 [==============================] - 0s 1ms/step - loss: 1.0676 - accura

In [ ]:
estimator_p.score(X,dummy_p)

236/236 [==============================] - 0s 931us/step - loss: 0.4625 - accuracy: 0.7957


0.7956886291503906

In [ ]:
test_data['breed_category'] = y_b['breed_category']
test_data.head()

,Date Difference,condition,color_encoded,Size,X1,X2,breed_category
0,4405.0,0.0,2,0.371751,0,7,1
1,175.0,1.0,36,0.004026,0,1,0
2,2000.0,1.0,2,0.098904,0,7,0
3,1148.0,1.0,2,0.024534,7,1,0
4,464.0,1.0,14,0.219532,0,7,0


#### Predict pet of test data :

In [ ]:
y_hat2=estimator_p.predict(test_data)
y_hat2

101/101 [==============================] - 0s 772us/step


array([2, 1, 2, ..., 2, 2, 2])

In [ ]:
y_p = pd.DataFrame(y_hat2)
y_p[0].unique()

array([2, 1, 3, 0])

In [ ]:
y_p.columns.values

array([0])

In [ ]:
cols = ['pet_category']
y_p.columns = cols
y_p.head()
y_p = y_p.replace(3,4)
y_p['pet_category']

0       2
1       1
2       2
3       2
4       2
       ..
8067    1
8068    2
8069    2
8070    2
8071    2
Name: pet_category, Length: 8072, dtype: int64

### Create final result data :

In [ ]:
final = pd.read_csv('test.csv')
final.head()

,pet_id,issue_date,listing_date,condition,color_type,length(m),height(cm),X1,X2
0,ANSL_75005,2005-08-17 00:00:00,2017-09-07 15:35:00,0.0,Black,0.87,42.73,0,7
1,ANSL_76663,2018-11-15 00:00:00,2019-05-08 17:24:00,1.0,Orange Tabby,0.06,6.71,0,1
2,ANSL_58259,2012-10-11 00:00:00,2018-04-02 16:51:00,1.0,Black,0.24,41.21,0,7
3,ANSL_67171,2015-02-13 00:00:00,2018-04-06 07:25:00,1.0,Black,0.29,8.46,7,1
4,ANSL_72871,2017-01-18 00:00:00,2018-04-26 13:42:00,1.0,Brown,0.71,30.92,0,7


In [ ]:
final.columns.values

array(['pet_id', 'issue_date', 'listing_date', 'condition', 'color_type',
       'length(m)', 'height(cm)', 'X1', 'X2'], dtype=object)

In [ ]:
final = final.drop(['issue_date', 'listing_date', 'condition', 'color_type',
       'length(m)', 'height(cm)', 'X1', 'X2'], axis=1)
final.head()

,pet_id
0,ANSL_75005
1,ANSL_76663
2,ANSL_58259
3,ANSL_67171
4,ANSL_72871


In [ ]:
final['breed_category'] = y_b['breed_category']
final['pet_category'] = y_p['pet_category']

final

,pet_id,breed_category,pet_category
0,ANSL_75005,1,2
1,ANSL_76663,0,1
2,ANSL_58259,0,2
3,ANSL_67171,0,2
4,ANSL_72871,0,2
...,...,...,...
8067,ANSL_66809,1,1
8068,ANSL_59041,1,2
8069,ANSL_60034,1,2
8070,ANSL_58066,0,2


### Save to csv :

In [ ]:
final.set_index('pet_id')
final.to_csv('Result5.csv')